In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 27.4 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Created wheel for pyspark: filename=pyspark-3.5.4-py2.py3-none-any.whl size=317849768 sha256=c7baad0585caeff2db3700178a14aa4d26a34bb46c06e51e0c357ee26869fc80
  Stored in directory: /Users/aaronrusk/Library/Caches/pip/wheels/8d/28/22/5dbae8a8714ef046cebd320d0ef7c92f5383903cf854c15c0c
Successfully built pyspark


In [2]:
from pyspark import SparkContext, SparkConf
from google.colab import drive

conf = SparkConf()
sc = SparkContext(conf=conf)

drive.mount('/content/drive')
lines = sc.textFile("/content/drive/My Drive/Datasets/fradulent_emails.txt")

# Splits lines into words and maps each word to (word, 1)
def processLine(line):
    words = line.split(" ")
    return [(word.lower(), 1) for word in words]

# Sum the counts for each word ie reduce by key
def sumWordCounts(a, b):
    return a + b

# Sort word counts in descending order to easily get top 20
def sortDesc(pair):
    return -pair[1]

myWords = lines.flatMap(processLine)
wordCounts = myWords.reduceByKey(sumWordCounts)
top20Words = wordCounts.takeOrdered(21, key=sortDesc)
# Got top 21 words because the most used "word" is : , which is not actually a word

for word, count in top20Words:
    print(str(word) + " : " + str(count))


ModuleNotFoundError: No module named 'google.colab'

In [39]:
sc.stop()

The most frequently used words in order are:
the, to, of, and, i, in, you, this, a, my, your, for, will, that, as, is, be, with, from, me

This count does not really give any indication that they are taken from phishing emails, they are simply 20 of what I would predict are some of the most commonly used words in the English language. To improve the word count tool, we could purposely omit these super common conjuctions, pronouns, etc. that would be common in any English text. We could have a list of these common words and compare the words we find in the emails, omitting the words that also appear in that list. Then what would be left are the most used uncommon words, which would be a better indicator of if an email is likely to be a scam.